# Проект для «Викишоп» с BERT

**Цель проекта:**

Обучить модель классифицировать комментарии на позитивные и негативные. В нашем распоряжении набор данных с разметкой о токсичности правок. Метрики качества *F1* должна быть не меньше 0.75

**Описание проекта:**

Интернет-магазин запускает новый сервис. Теперь пользователи могут редактировать и дополнять описания товаров, как в вики-сообществах. То есть клиенты предлагают свои правки и комментируют изменения других. Магазину нужен инструмент, который будет искать токсичные комментарии и отправлять их на модерацию. 

Данные находятся в файле `toxic_comments.csv`.
Столбец *text* в нём содержит текст комментария, а *toxic* — целевой признак.

<h1>Содержание<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Подготовка" data-toc-modified-id="Подготовка-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Подготовка</a></span></li><li><span><a href="#Обучение" data-toc-modified-id="Обучение-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Обучение</a></span></li><li><span><a href="#BERT" data-toc-modified-id="BERT-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>BERT</a></span></li><li><span><a href="#Выводы" data-toc-modified-id="Выводы-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Выводы</a></span></li><li><span><a href="#Чек-лист-проверки" data-toc-modified-id="Чек-лист-проверки-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Чек-лист проверки</a></span></li></ul></div>

In [38]:
import pandas as pd

from nltk.corpus import stopwords as nltk_stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords as nltk_stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, f1_score
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag


## Подготовка

In [39]:
df = pd.read_csv("/toxic_comments.csv")

corpus = df['text'].values.astype('U')


In [40]:
def preprocess_text(sen):

    # Remove punctuations and numbers
    sentence = re.sub('[^a-zA-Z]', ' ', sen)

    # Single character removal
    sentence = re.sub(r"\s+[a-zA-Z]\s+", ' ', sentence)

    # Removing multiple spaces
    sentence = re.sub(r'\s+', ' ', sentence)
    sentence = sentence.lower()
    return sentence
import re

df['clear_text'] = df.text.apply(lambda x: preprocess_text(x))
corpus = df.clear_text.values

In [41]:
m  = WordNetLemmatizer()
def lemmatize(text):
    return "".join(m.lemmatize(text))
df['lemm_text'] = df.text.apply(lambda x : lemmatize(x))

In [42]:
corpus = df.lemm_text.values



## Обучение

In [43]:
features = df['lemm_text']
target = df['toxic']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(features, target, test_size = 0.5)

In [45]:
nltk.download('stopwords')
stopwords = set(nltk_stopwords.words('english'))
count_tf_idf = TfidfVectorizer(stop_words = stopwords)

counter_tf_idf = count_tf_idf.fit(X_train)

X_train = counter_tf_idf.transform(X_train)
X_test = counter_tf_idf.transform(X_test)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Heraschanka\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## LogisticRegression

In [46]:
model = LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression()

In [47]:
predictions = model.predict(X_test)

In [48]:
from sklearn.metrics import f1_score, make_scorer
f1_score(y_test, predictions)

0.705303612605688

## DecisiontreeCLassifier

In [49]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
%%time
model = RandomForestClassifier()
model.fit(X_train, y_train)
predictions = model.predict(X_test)
f1_score(y_test, predictions)

Wall time: 3min 32s


0.7163077849721426

## LGBMClassifier

In [51]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import GridSearchCV

In [52]:
%%time
params = {'n_estimators':[150], 'n_jobs' : [-1], 'num_leaves' :[20,40], 'n_jobs' : [-1], 'random_state' : [42]}
grd = GridSearchCV(LGBMClassifier(), params)
grd.fit(X_train, y_train)

best_model = grd.best_estimator_
predictions = best_model.predict(X_test)
f1_score(y_test, predictions)

Wall time: 4min 12s


0.7691875570535777

Вывод:

В результате работы рассмотрения нескольких моделей классификации можно сделать вывод, что наилучший результат получается при использовании модели lightGBM.

## BERT

In [1]:
import torch
import transformers
import numpy as np
from tqdm import notebook
from sklearn.utils import shuffle


In [4]:
df = pd.read_csv('https://code.s3.yandex.net/datasets/toxic_comments.csv')
tokenizer = transformers.BertTokenizer.from_pretrained('bert-base-uncased')
model = transformers.BertModel.from_pretrained('bert-base-uncased')


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [5]:
#сбалансируем классы для модели
df_ones = df[df['toxic'] == 1].sample(2000, random_state=123)
df_zeros = df[df['toxic'] == 0].sample(2000, random_state=123)
new_df = shuffle(pd.concat([df_ones] + [df_zeros]))

In [6]:
tokenized = new_df['text'].apply(lambda x: tokenizer.encode(x[:512], add_special_tokens=True))

max_len = max(map(len, tokenized)) # вычисляем максимальное кол-во токенов в предложении

padded = np.array([i + [0]*(max_len - len(i)) for i in tokenized.values]) #добавляем 0 для строк, у которых длина меньше максимальной
attention_mask = np.where(padded != 0, 1, 0) # создаем маску, чтобы выделить значимые токены

batch_size = 20

In [ ]:
embeddings = []
for i in notebook.tqdm(range(padded.shape[0] // batch_size)):
        batch = torch.LongTensor(padded[batch_size*i:batch_size*(i+1)]) 
        attention_mask_batch = torch.LongTensor(attention_mask[batch_size*i:batch_size*(i+1)])
        
        with torch.no_grad():
            batch_embeddings = model(batch, attention_mask=attention_mask_batch)
        
        embeddings.append(batch_embeddings[0][:,0,:])
        

In [33]:
features = np.concatenate(embeddings) #полученные эмбеддинги будут признаками для обучения модели
target = new_df['toxic']

In [34]:
features_train, features_test, target_train, target_test = train_test_split(features, target, test_size=0.2, random_state=123)
model_lr = LogisticRegression(solver='liblinear', random_state=123)
model_lr.fit(features_train, target_train)


LogisticRegression(random_state=123, solver='liblinear')

In [35]:
pred_train = model_lr.predict(features_train)
pred_test = model_lr.predict(features_test)

In [36]:
print('F1 обучающей выборки: {:.2f}'.format(f1_score(target_train, pred_train)))
print('F1 тестовой выборки: {:.2f}'.format(f1_score(target_test, pred_test)))

F1 обучающей выборки: 0.95
F1 тестовой выборки: 0.86


## Выводы

Модель BERT справляется с классификацией твитов лучше всего.